Run notebook server with the following command:
```bash
poetry run python manage.py shell_plus --notebook
```
Dump data (needs to be done for DRN and OSM data):
```bash
poetry run python manage.py dumpdata composer composer.RouteLSABinding > db_<drn/osm>.json
```

In [7]:
def get_routes_with_bindings(data):
    routes = []
    bindings = []
    for entry in data:
        if entry["model"] == "composer.route":
            routes.append(entry)
        if entry["model"] == "composer.routelsabinding":
            bindings.append(entry)
    routes_with_bindings = {}
    for binding in bindings:
        if binding["fields"]["route"] not in routes_with_bindings:
            routes_with_bindings[binding["fields"]["route"]] = next(route for route in routes if route["pk"] == binding["fields"]["route"])
    return routes_with_bindings

def get_bindings_for_route_id(data, route_id):
    selected_lsa_ids = []
    for entry in data:
        if entry["model"] == "composer.routelsabinding" and entry["fields"]["route"] == route_id:
            selected_lsa_ids.append(entry["fields"]["lsa"])
    return selected_lsa_ids

from shapely.geometry import LineString as ShapelyLineString
from shapely import wkt
from routing.models import LSA
from django.conf import settings
from routing.matching.projection import project_onto_route

def get_distances(sg_ids, route_django_linestring):
    distances = set()
    for sg_id in sg_ids:
        sg = LSA.objects.get(pk=sg_id)
        sg_geometry = sg.geometry
        sg_geometry_meter = sg_geometry.transform(settings.METRICAL, clone=True)
        sg_geometry_meter_projected = project_onto_route(sg_geometry_meter, route_django_linestring)
        sg_geometry_meter_shapely = wkt.loads(sg_geometry_meter.wkt)
        sg_geometry_meter_projected_shapely = wkt.loads(sg_geometry_meter_projected.wkt)
        distances.add(sg_geometry_meter_shapely.hausdorff_distance(sg_geometry_meter_projected_shapely))
    return distances
    

# Import bindings

In [3]:
import json
import os
from django.conf import settings
from django.contrib.gis.geos import LineString
# Import DRN json file
f_drn = open('db_drn.json')
data_drn = json.load(f_drn)
routes_with_bindings_drn = get_routes_with_bindings(data_drn)
# Import OSM json file
f_osm = open('db_osm.json')
data_osm = json.load(f_osm)
routes_with_bindings_osm = get_routes_with_bindings(data_osm)

# Matching

In [8]:
from routing.matching.hypermodel import TopologicHypermodelMatcher
from routing.matching import get_matches

# DRN
tp_drn = set()
tp_drn_distances = set()
fp_drn = set()
fp_drn_distances = set()
fn_drn = set()
fn_drn_distances = set()
matchers_drn = [ TopologicHypermodelMatcher.from_config_file(f'config/topologic.hypermodel.drn.updated.json') ]
for route_id in routes_with_bindings_drn:
    routeGeometry = routes_with_bindings_drn[route_id]["fields"]["geometry"]
    srid = int(routeGeometry.split("=")[1].split(";")[0])
    points = [(float(point.split(" ")[0]), float(point.split(" ")[1])) for point in routeGeometry.replace("SRID=4326;LINESTRING (", "").replace(")","").split(", ")]
    linestring = LineString(points, srid=srid)
    matches = get_matches(linestring, matchers_drn)
    matched_lsas = [lsa.id for lsa in matches]
    selected_lsas = get_bindings_for_route_id(data_drn, routes_with_bindings_drn[route_id]["pk"])
    tp = set(matched_lsas) & set(selected_lsas)
    tp_drn_distances = get_distances(tp, linestring)
    tp_drn.update(tp)
    fp = set(matched_lsas) - set(selected_lsas)
    fp_drn_distances = get_distances(fp, linestring)
    fp_drn.update(fp)
    fn = set(selected_lsas) - set(matched_lsas)
    fn_drn_distances = get_distances(fn, linestring)
    fn_drn.update(fn)
    print(f"DRN TP: {len(tp_drn)}, FP: {len(fp_drn)}, FN: {len(fn_drn)}")
tp_drn_count = len(tp_drn)
fp_drn_count = len(fp_drn)
fn_drn_count = len(fn_drn)
precision_drn = tp_drn_count / (tp_drn_count + fp_drn_count) if tp_drn_count + fp_drn_count > 0 else 0
recall_drn = tp_drn_count / (tp_drn_count + fn_drn_count) if tp_tp_drn_countdrn + fn_drn_count > 0 else 0
f1_drn = 2 * precision_drn * recall_drn / (precision_drn + recall_drn) if precision_drn + recall_drn > 0 else 0
# OSM
tp_osm = set()
tp_osm_distances = set()
fp_osm = set()
fp_osm_distances = set()
fn_osm = set()
fn_osm_distances = set()
matchers_osm = [ TopologicHypermodelMatcher.from_config_file(f'config/topologic.hypermodel.osm.updated.json') ]
for route_id in routes_with_bindings_drn:
    routeGeometry = routes_with_bindings_drn[route_id]["fields"]["geometry"]
    srid = int(routeGeometry.split("=")[1].split(";")[0])
    points = [(float(point.split(" ")[0]), float(point.split(" ")[1])) for point in routeGeometry.replace("SRID=4326;LINESTRING (", "").replace(")","").split(", ")]
    linestring = LineString(points, srid=srid)
    matches = get_matches(linestring, matchers_osm)
    matched_lsas = [lsa.id for lsa in matches]
    selected_lsas = get_bindings_for_route_id(data_osm, routes_with_bindings_drn[route_id]["pk"])
    tp = set(matched_lsas) & set(selected_lsas)
    tp_osm_distances = get_distances(tp, linestring)
    tp_osm.update(tp)
    fp = set(matched_lsas) - set(selected_lsas)
    fp_osm_distances = get_distances(fp, linestring)
    fp_osm.update(fp)
    fn = set(selected_lsas) - set(matched_lsas)
    fn_osm_distances = get_distances(fn, linestring)
    fn_osm.update(fn)
    print(f"OSM TP: {len(tp_osm)}, FP: {len(fp_osm)}, FN: {len(fn_osm)}")
tp_osm_count = len(tp_osm)
fp_osm_count = len(fp_osm)
fn_osm_count = len(fn_osm)
precision_osm = tp_osm_count / (tp_osm_count + fp_osm_count) if tp_osm_count + fp_osm_count > 0 else 0
recall_osm = tp_osm_count / (tp_osm_count + fn_osm_count) if tp_osm_count + fn_osm_count > 0 else 0
f1_osm = 2 * precision_osm * recall_osm / (precision_osm + recall_osm) if precision_osm + recall_osm > 0 else 0

DRN TP: 12, FP: 1, FN: 0


KeyboardInterrupt: 

db_drn.json  matches/  routing_paper.ipynb


/code/backend/backend/notebooks/routing_paper/db_drn.json
